In [1]:
from geopy.distance import geodesic
import osmium
import os
import math
from scipy.spatial import KDTree
import numpy as np

out_file = "../osm/cost.osm"
input_file = '../osm/divided.osm'

In [2]:
def check_cost(length, lanes):
    return length / math.sqrt(lanes)

In [3]:
class OSMHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.node_locs = osmium.index.create_map("sparse_mem_array")
        self.loc_handler = osmium.NodeLocationsForWays(self.node_locs)
        self.ways = []
        self.builsings = []
        self.nods_way = []
        self.nods_build = []
        
    def node(self, n: osmium.Node):
        self.node_locs.set(n.id, n.location)

    def way(self, w: osmium.Way):
        if 'highway' in w.tags:
            total_length = 0.0
            lanes = float(w.tags.get('lanes', '2'))
            prev_node = None
            for n in w.nodes:
                #print((n.ref, n.location.lat, n.location.lon))
                self.nods_way.append((n.ref, n.location))
                if n.location.valid():
                    if prev_node is not None:
                        total_length += geodesic((prev_node.lat, prev_node.lon), (n.location.lat, n.location.lon)).meters
                    prev_node = n.location 
            cost = check_cost(total_length, lanes)
            new_way = osmium.osm.mutable.Way(w)
            new_way.tags = dict(w.tags)
            new_way.tags['cost'] = f"{cost:.2f}"
            self.ways.append((w.id, dict(w.tags), cost, list(w.nodes)))
        elif 'building' in w.tags:
            lat = 0
            lon = 0
            for n in w.nodes:
                self.nods_build.append((n.ref, n.location)) #useless
                lat = lat + n.lat
                lon = lon + n.lon
            lat = lat/len(w.nodes)
            lon = lon/len(w.nodes)
            self.builsings.append((w.id, dict(w.tags), lat, lon))


In [4]:
def save_modified_ways(ways, nodes, writer):
    for id, loc in nodes:
        node = osmium.osm.mutable.Node()
        node.id = id
        node.location = loc 
        writer.add_node(node)

    for way_id, tags, cost, nods in ways:
        new_way = osmium.osm.mutable.Way()
        new_way.id = way_id
        new_way.tags = tags
        new_way.tags['cost'] = f"{cost:.2f}"
        new_way.nodes = nods
        writer.add_way(new_way)
    writer.close()

In [6]:
def save_start_end_predict(buildings, nodes_way):
    # Создаем массив координат нод для KD-Tree
    node_coords = [(loc.lat, loc.lon) for (ref, loc) in nodes_way]
    node_tree = KDTree(node_coords)


    residential_buildings = []
    commercial_buildings = []

    capacity_residential = 1.0 
    capacity_commercial = 10.0 
    
    for id, tags, lat, lon in buildings:
        building_type = tags.get('building', "yes")
        building_coord = (lat, lon)
        capacity = 0
        
        if building_type in ['residential', 'house', 'apartments', 'dormitory', 'terrace', 'detached', 'bungalow']:
            match building_type:
                case 'apartments':
                    # print(tags.get('building:levels', -1), tags.get('length', -1), tags.get('width', -1) )
                    capacity = capacity_residential
                case 'residential','residential', 'house','dormitory', 'terrace', 'detached', 'bungalow':
                    # print("Обработка случая 2")
                    1 + 1            
            capacity = capacity_residential
            residential_buildings.append((id, building_coord, capacity))
        elif building_type in ['commercial', 'office', 'industrial', "retail", "store", "school", "hospital", 'garages', 'warehouse', 'service', 'kindergarten']:
            capacity = capacity_commercial
            commercial_buildings.append((id, building_coord, capacity))
            # print(building_type)
        elif building_type != 'yes':
            print(building_type)


In [8]:

if os.path.exists(out_file):
    os.remove(out_file)

writer = osmium.SimpleWriter(out_file)

try:
    h = OSMHandler()
    h.apply_file(input_file, locations=True)
    save_start_end_predict(h.builsings, h.nods_way)
    save_modified_ways(h.ways, h.nods_way, writer)
finally:
    writer.close()  